# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [5]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from scipy import stats

# Будем осуществлять работу с набором данных horse_data.csv о состоянии здоровья лошадей, испытывающих кишечные колики. 
# Ссылка на исходный файл
data_url = 'https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv'

# Загружаем исходные данные
data = pd.read_csv(data_url, header=None, na_values='?')  # для работы с пропусками делаем подмену ? на NaN

# Назначаем имена столбцов
data.columns = [
    "surgery", "age", "hospital_number", "rectal_temperature", "pulse", "respiratory_rate", 
    "temperature_of_extremities", "peripheral_pulse", "mucous_membranes", "capillary_refill_time", 
    "pain", "peristalsis", "abdominal_distension", "nasogastric_tube", "nasogastric_reflux", 
    "nasogastric_reflux_ph", "rectal_examination", "abdomen", "packed_cell_volume", "total_protein", 
    "abdominocentesis_appearance", "abdominocentesis_total_protein", "outcome", 
    "surgical_lesion", "type_of_lesion_1", "type_of_lesion_2", "type_of_lesion_3", "cp_data"
]

# Выбираем 8 столбцов с числовыми и категориальтными данными
selected_columns = [
    "rectal_temperature", "pulse", "respiratory_rate", "temperature_of_extremities", 
    "pain", "abdominal_distension", "packed_cell_volume", "outcome"
]
selected_data = data[selected_columns].copy() 

### Задание 1: Провести расчет базовых метрик для них, кратко описать результаты.
basic_stats = selected_data.describe(include='all').T  # Основные статистики для всех столбцов
modes = selected_data.mode().iloc[0]  # Моды для всех столбцов
basic_stats['mode'] = modes  # Добавим моду в таблицу

print("Базовые метрики для выбранных столбцов:\n", basic_stats)

### Задание 2: В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.
# Для определения выбросов используем Z-оценку
numeric_columns = ["rectal_temperature", "pulse", "respiratory_rate", "packed_cell_volume"]
outliers = {}

for col in numeric_columns:
    # Убираем все NaN 
    col_data = selected_data[col].dropna().reset_index(drop=True)
    
    # Вычисляем Z-оценку
    z_scores = np.abs(stats.zscore(col_data))
    
    # Находим выбросы по Z-оценке и сохраняем их в словарь
    outliers[col] = col_data[z_scores > 3]
    print(f"Выбросы в '{col}':\n", outliers[col])

# Гипотеза: выбросы могут быть вызваны ошибками измерений, аномалиями или экстремальными случаями.
# Решение: при наличии небольшого числа выбросов можно их удалить, иначе применить лог-трансформацию или замену на медиану.

### Задание 3: Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с п
# ропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.
missing_data = selected_data.isnull().sum()
print("\nКоличество пропусков по столбцам:\n", missing_data)

# Заполнение пропусков: числовые - медианой, категориальные - модой
for col in selected_data.columns:
    if selected_data[col].dtype == 'float64' or selected_data[col].dtype == 'int64':
        selected_data[col] = selected_data[col].fillna(selected_data[col].median())  # Числовые заполняем медианой
    else:
        selected_data[col] = selected_data[col].fillna(selected_data[col].mode()[0])  # Категориальные заполняем модой

print("\nДатафрейм без пропусков:\n", selected_data.head())

Базовые метрики для выбранных столбцов:
                             count       mean        std   min   25%   50%  \
rectal_temperature          240.0  38.167917   0.732289  35.4  37.8  38.2   
pulse                       276.0  71.913043  28.630557  30.0  48.0  64.0   
respiratory_rate            242.0  30.417355  17.642231   8.0  18.5  24.5   
temperature_of_extremities  244.0   2.348361   1.045054   1.0   1.0   3.0   
pain                        245.0   2.951020   1.307940   1.0   2.0   3.0   
abdominal_distension        244.0   2.266393   1.065131   1.0   1.0   2.0   
packed_cell_volume          271.0  46.295203  10.419335  23.0  38.0  45.0   
outcome                     299.0   1.551839   0.737187   1.0   1.0   1.0   

                             75%    max  mode  
rectal_temperature          38.5   40.8  38.0  
pulse                       88.0  184.0  48.0  
respiratory_rate            36.0   96.0  20.0  
temperature_of_extremities   3.0    4.0   3.0  
pain                     